In [1]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict, load_model, Metrics, save_model

In [15]:
process_train_data()
process_test_data()

train_data, test_data = load_split_processed_data()

2024-06-23 13:49:19 - Running pipeline: TRANSFORM_TRAIN (1:00:41)
2024-06-23 13:49:19 - 0. Running step: set_missings (1:00:41)


Processing train data...
-----------------


2024-06-23 13:49:22 - 1. Running step: reduce_mem_usage (1:00:43)
2024-06-23 13:49:22 - 2. Running step: feature_selection (1:00:44)
2024-06-23 13:49:22 - 3. Running step: merge_train_dpi_features (1:00:44)
2024-06-23 13:49:22 - 4. Running step: merge_train_bnum_features (1:00:44)


Mem. usage decreased to 231.19 Mb (75.0% reduction)


2024-06-23 13:49:23 - 5. Running step: merge_train_fe_features (1:00:44)
2024-06-23 13:49:23 - 6. Running step: merge_train_fe_total_features (1:00:45)
2024-06-23 13:49:23 - 7. Running step: feature_selection_new (1:00:45)
2024-06-23 13:49:24 - 8. Running step: merge_train_fe_slope_features (1:00:45)
2024-06-23 13:49:24 - 9. Running step: feature_selection_new (1:00:46)
2024-06-23 13:49:24 - 10. Running step: drop_high_correlation (1:00:46)
2024-06-23 13:49:46 - 12. Done (1:01:08)
2024-06-23 13:49:46 - Running pipeline: TRANSFORM_TEST (1:01:08)
2024-06-23 13:49:46 - 0. Running step: set_missings (1:01:08)


Features to drop: ['sms_in_cnt_max_total', 'voice_in_tar_dur_min_total', 'num_act_days_mea_mnt1', 'voice_in_roam_clc_mea_slope', 'com_num_part_min_slope', 'pay_p2p_in_sum_std_slope', 'voice_mts_out_nrest_partstd_slope', 'voice_in_short_part_max_slope', 'pay_sum_max_slope', 'conn_com_part_mea_slope', 'voice_out_fix_tar_dur_std_slope', 'pay_avg_std_slope', 'pay_max_max_slope', 'gprs_clc_max_slope', 'sms_roam_clc_std_slope', 'voice_mts_in_drest_part_std_slope', 'voice_mts_out_nrest_partmea_slope', 'pay_sum_mea_slope', 'voice_in_fix_tar_dur_std_slope', 'voice_out_short_part_std_slope', 'voice_in_kievstar_part_max_slope', 'clc_no_vas_roam_max_slope', 'voice_mts_in_nrest_part_max_slope', 'pay_avg_min_slope', 'voice_mts_out_nwork_partmax_slope', 'ama_volume_std_slope', 'voice_in_cmpttrs_avg_durstd_slope', 'non_accum_internet_vol_max_slope', 'abon_part_std_slope', 'pay_p2p_out_sum_min_slope', 'voice_mts_out_drest_partmax_slope', 'voice_mts_out_dwork_partmax_slope', 'pay_avg_max_slope', 'all_ro

2024-06-23 13:49:49 - 1. Running step: reduce_mem_usage (1:01:10)
2024-06-23 13:49:49 - 2. Running step: feature_selection (1:01:11)
2024-06-23 13:49:49 - 3. Running step: merge_test_dpi_features (1:01:11)
2024-06-23 13:49:50 - 4. Running step: merge_test_bnum_features (1:01:11)


Mem. usage decreased to 225.88 Mb (75.5% reduction)


2024-06-23 13:49:50 - 5. Running step: merge_test_fe_features (1:01:11)
2024-06-23 13:49:50 - 6. Running step: merge_test_fe_total_features (1:01:12)
2024-06-23 13:49:51 - 7. Running step: merge_test_fe_slope_features (1:01:12)
2024-06-23 13:49:51 - 9. Done (1:01:13)


-----------------

Shape: (150000, 1251)
Columns: Index(['num_act_days_mea_wk1', 'device_brand_other', 'num_act_days_min_mnt1',
       'active_ppm', 'num_act_days_min_mnt3', 'num_act_days_mea_mnt1',
       'loc_market_share', 'voice_mts_in_dwork_part_min_mnt1',
       'voice_in_kievstar_part_std_mnt1', 'device_type_smartphone',
       ...
       'accum_oth_dur_td_slope', 'data_3g_tv_cnt_td_slope',
       'voice_in_td_cnt_min_slope', 'abon_part_td_slope',
       'com_num_part_std_slope', 'sms_in_cnt_std_slope',
       'pay_p2p_out_sum_td_slope', 'pay_max_td_slope',
       'voice_in_short_part_td_slope', 'voice_out_cmpttrs_avg_dumin_slope'],
      dtype='object', length=1251)
Data saved


In [14]:
model_params = {
    "random_state": 42,
    "seed": 42,
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "feature_pre_filter": False,
    "lambda_l1": 2.740292203572801,
    "lambda_l2": 9.502614592851208,
    "learning_rate": 0.07586188568634152,
    "num_leaves": 93,
    "feature_fraction": 0.34049127666480095,
    "bagging_fraction": 0.8460348477289328,
    "max_depth": 10,
    "early_stopping_rounds": 88,
}


train_x = train_data.drop(columns="target")
train_y = train_data.target

valid_x = test_data.drop("target", axis=1)[train_x.columns]
valid_y = test_data.target

dtrain = lgb.Dataset(train_x, label=train_y)
dvalid = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)

model = lgb.train(
    model_params,
    dtrain,
    valid_sets=[dvalid],
)

X = test_data[train_x.columns]
y_true = test_data.target

y_pred_proba = model.predict(X, num_iteration=model.best_iteration)
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

Metrics().call(y_true, y_pred, y_pred_proba)

print("Columns count ", len(train_x.columns))
print("Columns: ", train_x.columns)


Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.71      0.32      0.44      9403

    accuracy                           0.95    150000
   macro avg       0.83      0.66      0.71    150000
weighted avg       0.94      0.95      0.94    150000

Columns count  349
Columns:  Index(['sms_in_cnt_std_total', 'casual_act_count', 'loc_market_share',
       'imei_mean_days_usage', 'SUM_of_Count_events_susp_app_sum',
       'voice_mts_in_dwork_part_mea_mnt3', 'num_act_days_std_total',
       'voice_out_td_cnt_std_total', 'MV_dou_net',
       'non_accum_internet_vol_td_total',
       ...
       'accum_oth_dur_td_slope', 'data_3g_tv_cnt_td_slope',
       'voice_in_td_cnt_min_slope', 'abon_part_td_slope',
       'com_num_part_std_slope', 'sms_in_cnt_std_slope',
       'pay_p2p_out_sum_td_slope', 'pay_max_td_slope',
       'voice_in_short_part_td_slope', 'voice_out_cmpttrs_avg

In [54]:
save_model(model, "LightGbmV2_new_total_features_p072_r032_090auc", list(train_x.columns))

Save LightGbmV2_new_total_features_p072_r032_090auc


'LightGbmV2_new_total_features_p072_r032_090auc'